In [8]:
#Importaciones
from functions import *
import pandas as pd

In [5]:
#Cargando base de datos
url_robberies = ("https://github.com/LuisHRF/Home-Insurance-Study-Project-/raw/main/01002%20(1).xlsx")
data_raw_robberies = pd.read_excel(url_robberies)
data_raw_robberies.head(10)

,,1. CONTRA LAS PERSONAS,1.2.-Lesiones,5.1.-Hurtos,5.2.-Robos con fuerza en las cosas,5.2.1.-Robos con fuerza en las cosas en el interior de vehículos,5.2.2.-Robos con fuerza en viviendas,5.2.3.-Robos con fuerza en establecimientos,5.3.-Robos con violencia o intimidación,5.3.1.-Robos con violencia en vía pública,5.3.2.-Robos con violencia en viviendas,5.3.3.-Robos con violencia en establecimientos
0,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Total Nacional,226761.0,118125.0,665622.0,272638.0,104988.0,84721.0,35072.0,64711.0,44406.0,3844.0,8238.0
2,Araba/Álava,1386.0,745.0,4184.0,1099.0,8.0,434.0,178.0,114.0,82.0,6.0,14.0
3,Albacete,1638.0,851.0,3771.0,1784.0,421.0,636.0,204.0,211.0,120.0,23.0,46.0
4,Alicante/Alacant,11416.0,5326.0,27014.0,12716.0,3723.0,6275.0,1241.0,1997.0,1161.0,245.0,343.0
5,Almería,4343.0,2517.0,6437.0,4193.0,1106.0,1602.0,399.0,657.0,397.0,67.0,81.0
6,Ávila,722.0,453.0,784.0,646.0,79.0,269.0,75.0,26.0,10.0,7.0,5.0
7,Badajoz,2516.0,1621.0,4844.0,2682.0,622.0,967.0,383.0,231.0,107.0,29.0,70.0
8,Balears (Illes),10159.0,4495.0,23009.0,6806.0,3195.0,1937.0,738.0,1519.0,994.0,82.0,228.0
9,Barcelona,16470.0,10527.0,141789.0,44688.0,22066.0,12042.0,5442.0,22792.0,17358.0,808.0,1787.0


In [4]:
data_clean = data_raw_robberies.dropna()

data_clean 

,,1. CONTRA LAS PERSONAS,1.2.-Lesiones,5.1.-Hurtos,5.2.-Robos con fuerza en las cosas,5.2.1.-Robos con fuerza en las cosas en el interior de vehículos,5.2.2.-Robos con fuerza en viviendas,5.2.3.-Robos con fuerza en establecimientos,5.3.-Robos con violencia o intimidación,5.3.1.-Robos con violencia en vía pública,5.3.2.-Robos con violencia en viviendas,5.3.3.-Robos con violencia en establecimientos
1,Total Nacional,226761.0,118125.0,665622.0,272638.0,104988.0,84721.0,35072.0,64711.0,44406.0,3844.0,8238.0
2,Araba/Álava,1386.0,745.0,4184.0,1099.0,8.0,434.0,178.0,114.0,82.0,6.0,14.0
3,Albacete,1638.0,851.0,3771.0,1784.0,421.0,636.0,204.0,211.0,120.0,23.0,46.0
4,Alicante/Alacant,11416.0,5326.0,27014.0,12716.0,3723.0,6275.0,1241.0,1997.0,1161.0,245.0,343.0
5,Almería,4343.0,2517.0,6437.0,4193.0,1106.0,1602.0,399.0,657.0,397.0,67.0,81.0
...,...,...,...,...,...,...,...,...,...,...,...,...
779,Zaragoza,5171.0,1988.0,14860.0,7904.0,1895.0,1756.0,1271.0,1531.0,1052.0,96.0,235.0
780,Ceuta,833.0,393.0,983.0,723.0,497.0,93.0,49.0,189.0,141.0,8.0,13.0
781,Melilla,1011.0,501.0,1043.0,774.0,427.0,156.0,79.0,194.0,171.0,1.0,8.0
782,En el extranjero,158.0,17.0,2427.0,674.0,142.0,19.0,51.0,177.0,138.0,5.0,6.0


In [6]:
unique_first_column = data_clean.iloc[:, 0].unique()

unique_first_column 

array(['    Total Nacional', '    Araba/Álava', '    Albacete',
       '    Alicante/Alacant', '    Almería', '    Ávila', '    Badajoz',
       '    Balears (Illes)', '    Barcelona', '    Burgos',
       '    Cáceres', '    Cádiz', '    Castellón/Castelló',
       '    Ciudad Real', '    Córdoba', '    Coruña (A)', '    Cuenca',
       '    Girona', '    Granada', '    Guadalajara', '    Gipuzkoa',
       '    Huelva', '    Huesca', '    Jaén', '    León', '    Lleida',
       '    Rioja (La)', '    Lugo', '    Madrid', '    Málaga',
       '    Murcia', '    Navarra', '    Ourense', '    Asturias',
       '    Palencia', '    Palmas (Las)', '    Pontevedra',
       '    Salamanca', '    Santa Cruz de Tenerife', '    Cantabria',
       '    Segovia', '    Sevilla', '    Soria', '    Tarragona',
       '    Teruel', '    Toledo', '    Valencia/València',
       '    Valladolid', '    Bizkaia', '    Zamora', '    Zaragoza',
       '    Ceuta', '    Melilla', '    En el extranjero',
   

In [35]:
# Calculate the variable block sizes

block_sizes = [55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55]
start = 1
year_ranges = {}

for i, size in enumerate(block_sizes):
    end = start + size - 1
    year = 2023 - i
    year_ranges[(start, end)] = year
    start = end + 2  

print(year_ranges)

{(1, 55): 2023, (57, 111): 2022, (113, 167): 2021, (169, 223): 2020, (225, 279): 2019, (281, 335): 2018, (337, 391): 2017, (393, 447): 2016, (449, 503): 2015, (505, 559): 2014, (561, 615): 2013, (617, 671): 2012, (673, 727): 2011, (729, 783): 2010}


In [ ]:
year_range = {
    (1, 55): 2023,
    (57, 111): 2022,
    (113, 167): 2021,
    (169, 223): 2020,
    (225, 279): 2019,
    (281, 335): 2018,
    (337, 391): 2017,
    (393, 447): 2016,
    (449, 503): 2015,
    (505, 559): 2014,
    (561, 615): 2013,
    (617, 671): 2012,
    (673, 727): 2011,
    (729, 783): 2010
}

data_clean_years = defi_years_per_block(data_clean, year_range)

data_clean_years.tail(10)

In [9]:
def verify_and_switch_datatypes(data_clean, assigned_types):

    # Verificar los tipos de datos actuales y convertir si es necesario
    for column, assigned_types in assigned_types.items():
        currently_type = data_clean[column].dtype
        
        if currently_type != assigned_types:
            try:
                data_clean[column] = data_clean[column].astype(assigned_types)
                print(f"Column '{column}' converted from {currently_type} a {assigned_types}.")
            except Exception as e:
                print(f"It was no possible to switch the '{column}' type into {assigned_types}: {e}")
        else:
            print(f"Column '{column}' is currently right typed as ({currently_type}).")
    
    return data_clean

In [ ]:
data_formated_types = verify_and_switch_datatypes(data_clean)